In [9]:
cd SMIT3.0

c:\SMIT3.0


เพิ่มการดึง Query ข้อมูลจาก Table AUDITResult, Classification_Finding 

In [10]:
from deep_translator import GoogleTranslator               
from app.Function import *                    
import pandas as pd                                        
import pyodbc                                              
import re                                                  
from sentence_transformers import SentenceTransformer, util

modelPath = "./Model/SentenceTransformer"

model = SentenceTransformer(modelPath)
 
connect_db = pyodbc.connect(Driver = "ODBC Driver 17 for SQL Server",
                            Server = "smitazure.database.windows.net",
                            Database = "SMIT3",
                            uid = 'smitadmin',
                            pwd = 'Abc12345',
                            Trusted_Connection = 'no')   

cursor = connect_db.cursor()

Cleansing_FindingDetails = pd.read_sql("SELECT * FROM [Cleansing_FindingDetails]", connect_db)

FindingNo = Cleansing_FindingDetails['FindingNo'].tolist()
FindingNo = ["-" if pd.isnull(x) else x for x in FindingNo]

Area = Cleansing_FindingDetails['Area'].tolist()
Area = ["-" if pd.isnull(x) else x for x in Area]

SubArea = Cleansing_FindingDetails['SubArea'].tolist()
SubArea = ["-" if pd.isnull(x) else x for x in SubArea]

Contractor = Cleansing_FindingDetails['Contractor'].tolist()
Contractor = ["-" if pd.isnull(x) else x for x in Contractor]

Tof = Cleansing_FindingDetails['TypeOfFinding'].tolist()
Tof = ["-" if pd.isnull(x) else x for x in Tof]

Topic = Cleansing_FindingDetails['Topic'].tolist()
Topic = ["-" if pd.isnull(x) else x for x in Topic]

Details = Cleansing_FindingDetails['Finding'].tolist()
Details = ["-" if pd.isnull(x) else x for x in Details]

CleansingDetails = Cleansing_FindingDetails['CleansingFinding'].tolist()
CleansingDetails = ["-" if pd.isnull(x) else x for x in CleansingDetails]

TranslateDetails = Cleansing_FindingDetails['TranslateFinding'].tolist()
TranslateDetails = ["-" if pd.isnull(x) else x for x in TranslateDetails]

-------------------------------------------------------------------------------------

In [5]:
Prepared_FindingNo = []
Prepared_Area = []
Prepared_SubArea = []
Prepared_Contractor = []
Prepared_Tof = []
Prepared_Topic = []
Prepared_Details = []
Prepared_Frequency = []
Prepared_CleansingDetails = []
Prepared_Translate_Details = []
Prepared_ListFindingNo = []
MostMatch = []

Encode_Translate_Cleansing_Safety_Audit = model.encode(TranslateDetails)

Cosine_Sim = util.cos_sim(Encode_Translate_Cleansing_Safety_Audit, Encode_Translate_Cleansing_Safety_Audit)


Size = len(Encode_Translate_Cleansing_Safety_Audit)

Sum = 0
for i in range(0, Size):
  Index_Most_Frequency = 0
  Count_Frequency = 0
  Max_Cosine = 0
  Index_Frequency = []
  if i not in MostMatch:
    if (Tof[i] == "Unsafe Condition"):
      for j in range(0, Size):
        if i != j:
          if Cosine_Sim[i][j] > 0.6 and Tof[j] == "Unsafe Condition":
              Count_Frequency += 1 
              Index_Frequency.append(j) 
              if Cosine_Sim[i][j] > Max_Cosine:
                Max_Cosine = Cosine_Sim[i][j]
                Index_Most_Frequency = j
        
    elif (Tof[i] == "Unsafe Action"):
      for j in range(0, Size):
        if i != j:
          if Cosine_Sim[i][j] > 0.6 and Tof[j] == "Unsafe Action": 
            Count_Frequency += 1 
            Index_Frequency.append(j) 
            if Cosine_Sim[i][j] > Max_Cosine:
              Max_Cosine = Cosine_Sim[i][j]
              Index_Most_Frequency = j

    elif (Tof[i] == "HNM"):
      for j in range(0, Size):
        if i != j:
          if Cosine_Sim[i][j] > 0.7 and Tof[j] == "HNM": 
            Count_Frequency += 1 
            Index_Frequency.append(j) 
            if Cosine_Sim[i][j] > Max_Cosine: 
              Max_Cosine = Cosine_Sim[i][j]
              Index_Most_Frequency = j

    elif (Tof[i] == "Near Miss"):
      for j in range(0, Size):
        if i != j:
          if Cosine_Sim[i][j] > 0.7 and Tof[j] == "Near Miss": 
            Count_Frequency += 1 
            Index_Frequency.append(j) 
            if Cosine_Sim[i][j] > Max_Cosine:
              Max_Cosine = Cosine_Sim[i][j]
              Index_Most_Frequency = j

    elif (Tof[i] == "Accident"):
      for j in range(0, Size):
        if i != j:
          if Cosine_Sim[i][j] > 0.7 and Tof[j] == "Accident": 
            Count_Frequency += 1 
            Index_Frequency.append(j) 
            if Cosine_Sim[i][j] > Max_Cosine:   
              Max_Cosine = Cosine_Sim[i][j]
              Index_Most_Frequency = j
              
    if Count_Frequency == 0:
      Index_Frequency.append(i)
      Count_Frequency = 1 
      Index_Most_Frequency = i  

    if (Index_Most_Frequency not in MostMatch) and Details[Index_Most_Frequency] not in Prepared_Details:
      Prepared_FindingNo.append(int(FindingNo[Index_Most_Frequency]))
      Prepared_Area.append(Area[Index_Most_Frequency])
      Prepared_SubArea.append(SubArea[Index_Most_Frequency])
      Prepared_Contractor.append(Contractor[Index_Most_Frequency])
      Prepared_Tof.append(Tof[Index_Most_Frequency])
      Prepared_Topic.append(Topic[Index_Most_Frequency])
      Prepared_Details.append(Details[Index_Most_Frequency])
      Prepared_Frequency.append(Count_Frequency)
      Prepared_CleansingDetails.append(CleansingDetails[Index_Most_Frequency])
      Prepared_Translate_Details.append(TranslateDetails[Index_Most_Frequency])
      Prepared_ListFindingNo.append(Index_Frequency)
      MostMatch.append(Index_Most_Frequency)

    if Index_Most_Frequency in MostMatch and (Details[Index_Most_Frequency] in Prepared_Details and Area[Index_Most_Frequency] in Prepared_Area):
      Index = MostMatch.index(Index_Most_Frequency)
      Temp_Prepared_ListFindingNo = Prepared_ListFindingNo[Index]+Index_Frequency
      Temp_Prepared_ListFindingNo = list(set(Temp_Prepared_ListFindingNo))
      Prepared_ListFindingNo[Index] = Temp_Prepared_ListFindingNo
      Prepared_Frequency[Index] = len(Temp_Prepared_ListFindingNo)

# print(MostMatch)

In [6]:
len(Prepared_FindingNo)

1990

In [7]:
connect_db = pyodbc.connect(Driver = "ODBC Driver 17 for SQL Server",
                            Server = "smitazure.database.windows.net",
                            Database = "SMIT3",
                            uid = 'smitadmin',
                            pwd = 'Abc12345',
                            Trusted_Connection = 'no')   


Prepared_Safety_Audit = list(zip(Prepared_FindingNo, Prepared_Area, Prepared_SubArea, 
                            Prepared_Contractor, Prepared_Tof, Prepared_Topic, Prepared_Details, 
                            Prepared_Frequency, Prepared_CleansingDetails, Prepared_Translate_Details))

cursor = connect_db.cursor()
Query = "INSERT INTO [Prepared_FindingDetails] VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);"

cursor.executemany(Query, Prepared_Safety_Audit)
connect_db.commit()

Head = ['FindingNo', 'Area', 'SubArea', 'Contractor', 'TypeOfFinding', 'Topic', 'Finding', 'Frequency', 'CleansingFinding', 'TranslateFinding']

with open('./SMIT_Data/Prepared_Safety_Audit.csv', 'w', newline='', encoding="utf-8") as f:
  write = csv.writer(f)
  write.writerow(Head)
  write.writerows(Prepared_Safety_Audit)

In [8]:
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer, util
import pandas as pd

modelPath = "./Model/SentenceTransformer"
# Test
model = SentenceTransformer(modelPath)

SafetyAudit = pd.read_csv('./SMIT_Data/Prepared_Safety_Audit.csv', encoding='utf-8')
SA_Details = SafetyAudit['Finding'].tolist()
SA_Area = SafetyAudit['Area'].tolist()
SA_Contractor = SafetyAudit['Contractor'].tolist()
SA_Tof = SafetyAudit['TypeOfFinding'].tolist()
SA_Topic = SafetyAudit['Topic'].tolist()
SA_Frequency = SafetyAudit['Frequency'].tolist()
SA_Details_Trans = SafetyAudit['TranslateFinding'].tolist()

Encode_Safey_Audit_Details = model.encode(SA_Details_Trans)
Encode_Safey_Audit_Details = np.array(Encode_Safey_Audit_Details)

with open('./SMIT_Data/Encode_SafeyAudit.pkl', 'wb') as files:
    pickle.dump(Encode_Safey_Audit_Details, files)